In [4]:
# multi
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os 

# step
# 1. Create folder and put all output folder, config.csv and machine.csv in it.
# 2. Revise code variable of date and input_scheduling_time to run.

date='_1.0.1_0402' #input
machines_df = pd.read_csv("./result"+ date +"/machines.csv")
config_df = pd.read_csv("./result"+ date +"/config.csv")[['peak_period','minute_threshold','weight_setup_times','weight_cr']]
std_df =pd.read_csv(("./result"+ date +"/config.csv"))["std_time"]

# time convert
input_scheduling_time= '2022-04-02 07:30' #input
std_time = int(time.mktime(time.strptime(std_df.iloc[0], "%y-%m-%d %H:%M")))
scheduling_time = int(time.mktime(time.strptime(input_scheduling_time, "%Y-%m-%d %H:%M")))
gap_time= round((scheduling_time - std_time)/60)
print(gap_time)
g_start_T= 0+ gap_time
g_end_T= 1440 + gap_time
rows=[]

for i in range(len(config_df.values)):
    print(i)
    result_df = pd.read_csv("./result"+ date +"/output_0-"+str(i)+"/result.csv")
    gap_record_df = pd.read_csv("./result"+ date +"/output_0-"+str(i)+"/record_gap.csv")

    ## Get rid of repeative value
    repeative_index = list(result_df[result_df.duplicated("lot_number",keep=False)].index)
    delete_index =[]
    for i in range(len(repeative_index)):
        if result_df.iloc[repeative_index[i]].at['oper'] !=0: # remove overlap job #刪掉非機台上有被排上的貨
            if result_df.iloc[repeative_index[i]].at['process_time'] !=0: # remove overlap job
                delete_index.append(repeative_index[i])
            
    no_NA_df = result_df.drop(index=delete_index)
    no_NA_df.reset_index(inplace=True, drop=True)
    #----------------------------

    #----------------------------
    def get_setup(gap_record_df):
        oneday_gap_record_df= gap_record_df[(gap_record_df.start_time>=g_start_T) & (gap_record_df.start_time<=g_end_T)]
        #oneday_gap_record_df= gap_record_df[(gap_record_df.end_time>=g_start_T) & (gap_record_df.start_time<=g_end_T)]
        return len(oneday_gap_record_df.index)
        

    # utilization
    def get_utilization(no_NA_df):

        #oneday_result=no_NA_df[no_NA_df["start_time"]<= g_end_T]  # for 空機&稼動率
        oneday_result = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.start_time<=g_end_T)]
        entities=oneday_result.groupby("entity") #within one day
        entity_key_list= list(entities.groups.keys())
        total_processing_time=0
        total_processing_time_list=[]
        utilization_rate=0
        utilization_rate_dic={}
        utilization_rate_list=[]
        ## 一日空機
        count=0
        a = list(entities.size())
        for i in range(len(a)):
            if a[i]==1:
                count+=1

        #-----------------------------------------------------------------

        for i in range(len(entity_key_list)): #len(entity_key_list)
            key= entity_key_list[i]
            if i !=0: # first not in 
                total_processing_time_list.append(total_processing_time)
                #total_processing_time_list.append((entity_key_list[i-1],total_processing_time)) # test
                utilization_rate_dic[entity_key_list[i-1]] = round(total_processing_time/1440*100,2)
                total_processing_time=0

            for j in range(len(entities.groups[entity_key_list[i]])):
                row_index=entities.groups[key][j]
                if float(no_NA_df.iloc[row_index].at['start_time']) < g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) >g_end_T : # 
                    processing_time = 1440
                elif float(no_NA_df.iloc[row_index].at['start_time']) < g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) <=g_end_T: # exceed one day
                    processing_time = float(no_NA_df.iloc[row_index].at['end_time']) - g_start_T 
                elif float(no_NA_df.iloc[row_index].at['start_time']) >= g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) >g_end_T: # exceed one day
                    processing_time = g_end_T - float(no_NA_df.iloc[row_index].at['start_time'])
                else: # normal 
                    processing_time = float(no_NA_df.iloc[row_index].at['end_time']) - float(no_NA_df.iloc[row_index].at['start_time']) 

                total_processing_time += processing_time

            if i == len(entity_key_list)-1: # final one
                total_processing_time_list.append(total_processing_time)
                ## test it
                #total_processing_time_list.append((entity_key_list[i],total_processing_time)) # test
                utilization_rate_dic[entity_key_list[i]] = round(total_processing_time/1440*100,2)

        ratio= sum(total_processing_time_list)/(1440*len(machines_df.index))

        return "{:.2%}".format(ratio)
        

    # 一日產量
    def get_outplan(no_NA_df):
        outplan_df = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.end_time<=g_end_T)]
        outplan_df.reset_index(inplace=True, drop=True)
        outplan = 0
        for i in range(len(outplan_df.index)):
            outplan += outplan_df.iloc[i].at['qty']
        if outplan>1000:
            #print("產量:",round(outplan/1000),"K")
            return str(round(outplan/1000))+'K' 
        else:
            #print("產量:",outplan)
            return outplan 

    # 排上機
    def get_running_machines(no_NA_df,machines_df):
        outplan_df = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.start_time<=g_end_T)]
        outplan_df.reset_index(inplace=True, drop=True)    
        entities=outplan_df.groupby("entity") #within one day
        entity_key_list= list(entities.groups.keys())

        # total machine
        entities_2=machines_df.groupby("ENTITY") #within one day
        entity_key_list_2 = list(entities_2.groups.keys())

        # print no caculate utilization entity
        not_in_one_day = list(set(entity_key_list_2) - set(entity_key_list))
        #print("Not running:",sorted(not_in_one_day))
        return "{}/{}".format(len(entity_key_list),len(entity_key_list_2)) 

    rows.append([get_setup(gap_record_df),get_utilization(no_NA_df),get_outplan(no_NA_df),get_running_machines(no_NA_df,machines_df)])

indicator_df = pd.DataFrame(rows,columns=['Setup','Utilization','Outplan','Running_machines'])
final_df = pd.concat([config_df,indicator_df],axis=1)
# write to csv
final_df.to_csv("indicator_"+ date +".csv", index=False ,na_rep=0,encoding='utf_8_sig') 
final_df

-35
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


,peak_period,minute_threshold,weight_setup_times,weight_cr,Setup,Utilization,Outplan,Running_machines
0,0.0,4320,10000000000,0.0001,102,50.67%,3716K,783/967
1,0.1,4320,10000000000,0.0001,116,51.33%,3730K,784/967
2,0.2,4320,10000000000,0.0001,105,51.25%,3689K,784/967
3,0.3,4320,10000000000,0.0001,114,52.16%,3752K,787/967
4,0.4,4320,10000000000,0.0001,123,53.09%,3811K,789/967
5,0.5,4320,10000000000,0.0001,117,53.19%,3715K,792/967
6,0.6,4320,10000000000,0.0001,132,53.70%,3720K,791/967
7,0.7,4320,10000000000,0.0001,137,54.47%,3731K,794/967
8,0.9,4320,10000000000,0.0001,162,56.14%,3755K,797/967
9,1.0,4320,10000000000,0.0001,159,55.94%,3761K,796/967


In [5]:
# Single 
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os 
import plotly.express as px

#Step
# 1. Choose one output folder and put machine.csv in it.
# 2. Revise code variable of filename and input_scheduling_time to run it.

# read file
filename= '/output_0-19' #input
machines_df = pd.read_csv("./"+ filename +"/machines.csv")
config_df = pd.read_csv("./"+ filename +"/config.csv")[['peak_period','minute_threshold','weight_setup_times','weight_cr']]
std_df =pd.read_csv(("./"+ filename +"/config.csv"))["std_time"]
result_df = pd.read_csv("./"+ filename +"/result.csv")
gap_record_df = pd.read_csv("./"+ filename +"/record_gap.csv")

# time convert
input_scheduling_time= '2022-05-09 07:30' #input
std_time = int(time.mktime(time.strptime(std_df.iloc[0], "%y-%m-%d %H:%M")))
scheduling_time = int(time.mktime(time.strptime(input_scheduling_time, "%Y-%m-%d %H:%M")))
gap_time= round((scheduling_time - std_time)/60)
print(gap_time)
g_start_T= 0+ gap_time
g_end_T= 1440 + gap_time
rows=[]

# Get rid of repeative value
repeative_index = list(result_df[result_df.duplicated("lot_number",keep=False)].index)   #重複值包含空值!
delete_index =[]
for i in range(len(repeative_index)):
    if result_df.iloc[repeative_index[i]].at['oper'] !=0: # remove overlap job #刪掉非機台上有被排上的貨
        if result_df.iloc[repeative_index[i]].at['process_time'] !=0: # remove overlap job #刪掉非機台上有被排上的貨
            delete_index.append(repeative_index[i])
        
no_NA_df = result_df.drop(index=delete_index)
no_NA_df.reset_index(inplace=True, drop=True)

# Test range

#----------------------------
def get_setup(gap_record_df):
    oneday_gap_record_df= gap_record_df[(gap_record_df.start_time>=g_start_T) & (gap_record_df.start_time<=g_end_T)]
    #oneday_gap_record_df= gap_record_df[(gap_record_df.end_time>=g_start_T) & (gap_record_df.start_time<=g_end_T)]
    return len(oneday_gap_record_df.index)
    
# 改機
def get_setup_o(no_NA_df):

    sorted_result_df=no_NA_df[no_NA_df["start_time"]<= g_end_T].sort_values(by=['entity', 'end_time']) #within one day
    setup_dict ={}
    times_info_list=[]
    entity_name= sorted_result_df['entity'].iloc[0] # default
    bd_id = sorted_result_df['bd_id'].iloc[0]       # default
    total_setup_times=0

    for i in range(len(sorted_result_df.index)): # serach for all data
        if entity_name == sorted_result_df['entity'].iloc[i]:  # If entity is same, need to judge whether bd_id is same. Else, record new entity(key)

            if bd_id != sorted_result_df['bd_id'].iloc[i]: # if bd_id is different need to record
                setup_start=sorted_result_df['end_time'].iloc[i-1]  #the endT of previous and convert to date
                setup_end=sorted_result_df['start_time'].iloc[i]  #the startT of current and convert to date
                # record
                if setup_start >= g_start_T: #**
                    times_info_list.append(str(setup_start)+' > '+str(setup_end)) 
                    total_setup_times +=1
                bd_id = sorted_result_df['bd_id'].iloc[i]

        else: 
            #add to dict
            setup_dict[entity_name]=times_info_list
            # if len(times_info_list)!=0:
            #     print(entity_name," ",len(times_info_list))
            #next entity
            entity_name = sorted_result_df['entity'].iloc[i]
            bd_id = sorted_result_df['bd_id'].iloc[i]
            times_info_list=[]
    # test it
    #print(setup_dict)  

    return total_setup_times
    

# utilization
def get_utilization(no_NA_df):
    #oneday_result=no_NA_df[no_NA_df["start_time"]<= g_end_T]  # for 空機&稼動率
    oneday_result = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.start_time<=g_end_T)]
    entities=oneday_result.groupby("entity") #within one day
    entity_key_list= list(entities.groups.keys())
    total_processing_time=0
    total_processing_time_list=[]
    utilization_rate=0
    utilization_rate_dic={}
    utilization_rate_list=[]
    ## 一日空機
    count=0
    a = list(entities.size())
    for i in range(len(a)):
        if a[i]==1:
            count+=1

    #-----------------------------------------------------------------

    for i in range(len(entity_key_list)): #len(entity_key_list)
        key= entity_key_list[i]
        if i !=0: # first not in 
            total_processing_time_list.append(total_processing_time)
            #total_processing_time_list.append((entity_key_list[i-1],total_processing_time)) # test
            utilization_rate_dic[entity_key_list[i-1]] = round(total_processing_time/1440*100,2)
            total_processing_time=0

        for j in range(len(entities.groups[entity_key_list[i]])):
            row_index=entities.groups[key][j]
            if float(no_NA_df.iloc[row_index].at['start_time']) < g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) >g_end_T : # 
                processing_time = 1440
            elif float(no_NA_df.iloc[row_index].at['start_time']) < g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) <=g_end_T: # exceed one day
                processing_time = float(no_NA_df.iloc[row_index].at['end_time']) - g_start_T 
            elif float(no_NA_df.iloc[row_index].at['start_time']) >= g_start_T and float(no_NA_df.iloc[row_index].at['end_time']) >g_end_T: # exceed one day
                processing_time = g_end_T - float(no_NA_df.iloc[row_index].at['start_time'])
            else: # normal 
                processing_time = float(no_NA_df.iloc[row_index].at['end_time']) - float(no_NA_df.iloc[row_index].at['start_time']) 

            total_processing_time += processing_time

        if i == len(entity_key_list)-1: # final one
            total_processing_time_list.append(total_processing_time)
            ## test it
            #total_processing_time_list.append((entity_key_list[i],total_processing_time)) # test
            utilization_rate_dic[entity_key_list[i]] = round(total_processing_time/1440*100,2)

    #print(len(total_processing_time_list))                                             # test
    #print(utilization_rate_dic)  

    # plot 
    u_df = pd.DataFrame(list(utilization_rate_dic.items()),columns=['Entity', 'Utilization_rate'])
    # fig = px.bar(u_df, x='Entity', y='Utilization_rate', title='Distribution')
    # fig.show()
    ratio= sum(total_processing_time_list)/(1440*len(machines_df.index))

    return "{:.2%}".format(ratio),u_df
    

# 一日產量
def get_outplan(no_NA_df):
    outplan_df = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.end_time<=g_end_T)]
    outplan_df.reset_index(inplace=True, drop=True)
    outplan = 0
    for i in range(len(outplan_df.index)):
        outplan += outplan_df.iloc[i].at['qty']
    if outplan>1000:
        #print("產量:",round(outplan/1000),"K")
        return str(round(outplan/1000))+'K' 
    else:
        #print("產量:",outplan)
        return outplan 

# 排上機
def get_running_machines(no_NA_df,machines_df):
    outplan_df = no_NA_df[(no_NA_df.end_time>=g_start_T) & (no_NA_df.start_time<=g_end_T)]
    outplan_df.reset_index(inplace=True, drop=True)    
    entities=outplan_df.groupby("entity") #within one day
    entity_key_list= list(entities.groups.keys())

    # total machine
    entities_2=machines_df.groupby("ENTITY") #within one day
    entity_key_list_2 = list(entities_2.groups.keys())

    # print no caculate utilization entity
    not_in_one_day = list(set(entity_key_list_2) - set(entity_key_list))
    print("Not running:",sorted(not_in_one_day))
    return "{}/{}".format(len(entity_key_list),len(entity_key_list_2)), not_in_one_day

utilization_data = get_utilization(no_NA_df)
get_running_machines_data = get_running_machines(no_NA_df,machines_df)

rows.append([get_setup(gap_record_df),utilization_data[0],get_outplan(no_NA_df),get_running_machines_data[0]])

indicator_df = pd.DataFrame(rows,columns=['Setup','Utilization','Outplan','Running_machines'])
final_df = pd.concat([config_df,indicator_df],axis=1)
# write to csv
#final_df.to_csv("indicator_"+ filename +".csv", index=False ,na_rep=0,encoding='utf_8_sig') 
#final_df
# get_utilization(no_NA_df)[1]

not_run_dic={}
for i in get_running_machines_data[1]:
    not_run_dic[i]=0
not_df = pd.DataFrame(list(not_run_dic.items()),columns=['Entity', 'Utilization_rate'])

aaa = pd.concat([utilization_data[1], not_df], join="inner")
fig = px.bar(aaa.sort_values(by=['Entity']), x='Entity', y='Utilization_rate', title='Distribution')
fig.show()
#fig.write_html("utilization.html")


-36
Not running: ['BB211', 'BB212', 'BB213', 'BB214', 'BB215', 'BB217', 'BB219', 'BB220', 'BB221', 'BB222', 'BB223', 'BB225', 'BB226', 'BB227', 'BB228', 'BB229', 'BB230', 'BB231', 'BB232', 'BB233', 'BB234', 'BB235', 'BB441', 'BB442', 'BB443', 'BB444', 'BB445', 'BB446', 'BB449', 'BB450', 'BB452', 'BB453', 'BB454', 'BB455', 'BB456', 'BB459', 'BB460', 'BB465', 'BB467', 'BB468', 'BB469', 'BB470', 'BB473', 'BB474', 'BB475', 'BB477', 'BB480', 'BB482', 'BB483', 'BB484', 'BB486', 'BB487', 'BB488', 'BB489', 'BB490', 'BB496', 'BB497', 'BB499', 'BB500', 'BB501', 'BB503', 'BB506', 'BB507', 'BB508', 'BB509', 'BB510', 'BB513', 'BB514', 'BB515', 'BB516', 'BB517', 'BB518', 'BB519', 'BB521', 'BB522', 'BB523', 'BB525', 'BB526', 'BB527', 'BB528', 'BB529', 'BB530', 'BB531', 'BB532', 'BB533', 'BB534', 'BB535', 'BB537', 'BB538', 'BB539', 'BB541', 'BB542', 'BB543', 'BB544', 'BB545', 'BB547', 'BB549', 'BB553', 'BB554', 'BB556', 'BB558', 'BB559', 'BB560', 'BB561', 'BB562', 'BB564', 'BB565', 'BB566', 'BB567', '

In [ ]:
import csv
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os 

# Step
# 1. Revise input info to run it

# Read file
input_scheduling_time= '2022-05-09 07:30'
result_df = pd.read_csv("./result_0417_new/output_0-4/result.csv")


ini_time_stamp = int(time.mktime(time.strptime(input_scheduling_time, "%Y-%m-%d %H:%M")))
## Get rid of repeative and NA value
repeative_index = list(result_df[result_df.duplicated("lot_number",keep=False)].index)
delete_index =[]
for i in range(len(repeative_index)):
    if result_df.iloc[repeative_index[i]].at['oper']==0: # remove lot number equals to NA by oper
        delete_index.append(repeative_index[i])
    elif result_df.iloc[repeative_index[i]].at['process_time'] !=0: # keep jobs runnning on machine by process_time
        delete_index.append(repeative_index[i])
        
no_NA_df = result_df.drop(index=delete_index) # remove  
no_NA_df.reset_index(inplace=True, drop=True)  # reset index

# 1. Group the result.csv by entity 
# 2. Use group key(entity) to find which lot processing on this entity, and plot each entity's lots by order.
# 3. Using plotly.express.timeline to plot gantt, which need the information of below data.(need using time stamp to convert date )

def plot_gantt_by_order(no_NA_df):

    def createFolder(directory):
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory) 

    # initizlize
    plot_num = 50 #each time plot 50 entities
    entities= no_NA_df.groupby("entity") # group by entity
    entity_key_list= list(entities.groups.keys()) # key list
    color_discrete_sequence_list=['#F0F8FF','#7FFFD4','#F0FFFF','#F5F5DC','#FFE4C4',    # add color cycle
                '#000000','#0000FF','#8A2BE2','#A52A2A','#DEB887','#5F9EA0',
                '#7FFF00','#D2691E','#FF7F50','#6495ED','#FFF8DC','#DC143C','#00FFFF',
                '#00008B','#008B8B','#B8860B','#A9A9A9','#006400','#BDB76B','#8B008B',
                '#556B2F','#FF8C00','#9932CC','#8B0000','#E9967A','#8FBC8F','#483D8B',
                '#2F4F4F','#00CED1','#9400D3','#FF1493','#00BFFF','#696969','#1E90FF',
                '#B22222','#FFFAF0','#228B22','#FF00FF','#DCDCDC','#F8F8FF','#FFD700',
                '#DAA520','#808080','#008000','#ADFF2F','#F0FFF0','#FF69B4','#CD5C5C',
                '#4B0082','#FFFFF0','#F0E68C','#E6E6FA','#FFF0F5','#7CFC00','#FFFACD',
                '#ADD8E6','#F08080','#E0FFFF','#FAFAD2','#90EE90','#D3D3D3','#FFB6C1',
                '#FFA07A','#20B2AA','#87CEFA','#778899','#B0C4DE','#FFFFE0','#00FF00',
                '#32CD32','#FAF0E6','#FF00FF','#800000','#66CDAA','#0000CD','#BA55D3',
                '#9370DB','#3CB371','#7B68EE','#00FA9A','#48D1CC','#C71585','#191970',
                '#F5FFFA','#FFE4E1','#FFE4B5','#FFDEAD','#000080','#FDF5E6','#808000',
                '#6B8E23','#FFA500','#FF4500','#DA70D6','#EEE8AA','#98FB98','#AFEEEE',
                '#DB7093','#FFEFD5','#FFDAB9','#CD853F','#FFC0CB','#DDA0DD','#B0E0E6',
                '#800080','#FF0000','#BC8F8F','#4169E1','#8B4513','#FA8072','#FAA460',
                '#2E8B57','#FFF5EE','#A0522D','#C0C0C0','#87CEEB','#6A5ACD','#708090',
                '#FFFAFA','#00FF7F','#4682B4','#D2B48C','#008080','#D8BFD8','#FF6347',
                '#40E0D0','#EE82EE','#F5DEB3','#FFFFFF','#F5F5F5','#FFFF00','#9ACD32']  
    df=[] #plot need dataframe
    plt_count=0 # record the times current plot
    foldername = './plot_by_order' + '/' # folder which store result picture
    createFolder(foldername)  

    #plot
    for i in range(len(entity_key_list)): 
        key= entity_key_list[i]
        plt_count+=1
        for j in range(len(entities.groups[key])):
            row_index=entities.groups[key][j]
            Lot_id =no_NA_df.iloc[row_index].at['lot_number'] 
            Entity =no_NA_df.iloc[row_index].at['entity']
            start_ts =  time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ini_time_stamp+no_NA_df.iloc[row_index].at['start_time']*60)) 
            end_ts =  time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ini_time_stamp+no_NA_df.iloc[row_index].at['end_time']*60)) 
            Sets = str(no_NA_df.iloc[row_index].at['part_no'])+ "/"+ str(no_NA_df.iloc[row_index].at['part_id'])
            Bd_id= no_NA_df.iloc[row_index].at['bd_id']
            # create plot needed dateframe
            df.append(dict(Lot_ID= Lot_id, Start=start_ts,Finish=end_ts,Entity=Entity,Sets=Sets,Bd_Id=Bd_id))
        if(plt_count==plot_num) or i==len(entity_key_list)-1: # consider final part
        #show figure
            fig = px.timeline(df, x_start="Start", x_end="Finish", y="Entity", color="Bd_Id",color_discrete_sequence=color_discrete_sequence_list,text="Lot_ID",hover_name="Sets")  
            fig.update_yaxes(categoryorder="category descending") # sort by
            fig.update_traces(textposition='inside',marker_line_color='rgb(8,48,107)')
            fig.write_html(foldername +df[0]['Entity']+"_" +df[-1]['Entity']+".html")
            # reset
            df=[]
            plt_count=0

    pass


plot_gantt_by_order(no_NA_df)
